In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import cv2
from pywt import dwt2
import pywt
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Train Data

In [ ]:
# Each cars data containing hyundai and kia Sportage images are read and resized and stored respective arrays (hyundai_img and kia_img)
hyundai_train  = glob.glob("../input/new-train-dataset-cars/TRAIN DATASET/hyundai/*.jpg")


kia_sportage_train  =glob.glob("../input/new-train-dataset-cars/TRAIN DATASET/kia_sportage/*.jpg")
hyundai_img= []
kia_img =  []
for img in hyundai_train:
    n= cv2.imread(img)
    n= cv2.resize(n,(350,350))
    hyundai_img.append(n)
array1 = np.asarray(hyundai_img)
print(array1.shape)


for img1 in kia_sportage_train:
    n= cv2.imread(img1)
    n= cv2.resize(n,(350,350))
    kia_img.append(n)
array2 = np.asarray(kia_img)
print(array2.shape)


In [ ]:
# here the array is reshaped 
hyundai_reshaped = array1.reshape(len(array1),(350*350*3))
kia_reshaped = array2.reshape(len(array2),(350*350*3))

# the labels are genereted for hyundai and kia train data
# label is for hyundai and label1 is for kia sportage
label = np.zeros(len(array1))
label1 = np.ones(len(array2))

hyundai_reshaped.shape , kia_reshaped ,label , label1


In [ ]:
# the dataframe is formed of reshaped array 
df1 = pd.DataFrame(hyundai_reshaped)
df2 = pd.DataFrame(kia_reshaped)

df1["label"] = label.astype(int)
df2["label"] = label1.astype(int)

# the df1 and df2 are concatinated
result_train = pd.concat([df1, df2], ignore_index=True, sort=False )
result_train = result_train.sample(frac =1) # data is shuffled
result_train # train data is formed


# Test Data

In [ ]:
hyundai_test_img = glob.glob("../input/2nd-mid-test-data/2nd_mid_test_data/tucson/*.jpg")

hyundai_image= []

for img2 in hyundai_test_img:
    n= cv2.imread(img2)
    n= cv2.resize(n,(350,350))
    hyundai_image.append(n)
array4 = np.asarray(hyundai_image)
# array4.shape
hyundai_test_reshaped = array4.reshape(len(array4),(350*350*3))
hyundai_test_reshaped.shape ,array4.shape

df3= pd.DataFrame(hyundai_test_reshaped)
label3= np.zeros(len(array4))
df3["label"] = label3.astype(int)




In [ ]:
kia_sportage_test  = glob.glob("../input/2nd-mid-test-data/2nd_mid_test_data/Sportage/*.jpg")


kia_test=  []
for img3 in kia_sportage_test:
    n= cv2.imread(img3)
    n= cv2.resize(n,(350,350))
    kia_test.append(n)
array5 = np.asarray(kia_test)
                    
kia_test_reshaped = array5.reshape(len(array5),(350*350*3))
kia_test_reshaped.shape ,array5.shape

df4= pd.DataFrame(kia_test_reshaped)
label4= np.ones(len(array5))
df4["label"] = label4.astype(int)

result_test = pd.concat([df3, df4], ignore_index=True, sort=False )
result_test  = result_test.sample(frac=1)



In [ ]:
Xtrain,ytrain = result_train.loc[: , result_train.columns !='label'] ,result_train.label


# KNN model

In [ ]:
# the KNN model is initiated 
model=KNeighborsClassifier(n_neighbors=5)
model.fit(Xtrain, ytrain)

In [ ]:
# model predicts on Xtest data
Xtest , ytest  =  result_test.loc[: , result_test.columns !='label'] ,result_test.label


test_results= model.predict(Xtest)
test_results

# KNN accuracy

In [ ]:
# accuracy is printed of test data
from sklearn.metrics import accuracy_score
accuracy= accuracy_score (test_results, ytest)
accuracy

# Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(model , Xtest, ytest)


# Validation curve

In [ ]:
# here the validation curve is ploted 
# it calculates the accuracy vs neighbors. 
# this makes easy to set the value of k so that model good works
import matplotlib.pyplot as plt
total_train_accuracy= []
total_test_accuracy  = []
neighbors=  np.arange(1,15)
 
for i in range(14):
    knn= KNeighborsClassifier(n_neighbors=i+1)
    knn.fit(Xtrain, ytrain)
    train_accuracy =knn.score(Xtrain, ytrain)
    test_accuracy = knn.score(Xtest, ytest)
    
    total_train_accuracy.append(train_accuracy)
    total_test_accuracy.append(test_accuracy)
plt.title('k-NN: Varying number of neighbors')
plt.plot(neighbors, total_test_accuracy, label= 'Testing Accuracy')
plt.plot(neighbors, total_train_accuracy,  label ='Train Accuracy')
plt.legend()
plt.xlabel('number of neighbors')
plt.ylabel('accuracy')
plt.show()


# Logistic Regression

In [ ]:
# the logistic regrassion is model is initiated and score is printed
model_log = LogisticRegression()
model_log.fit(Xtrain, ytrain)
model_log.score(Xtest, ytest)


In [ ]:
#Analyzing model training and testing accuracy using Logistic Linear Regression model:
test_score = []
train_score  = []
arr= 5*np.arange(5)
for i in arr:
    model_Log = LogisticRegression(max_iter = i)
    model_Log.fit(Xtrain ,ytrain)
    test_score.append(model_Log.score(Xtest, ytest))
    train_score.append(model_Log.score(Xtrain, ytrain))
plt.xlabel("iteration")
plt.ylabel("accuracy")
plt.title("logistic regression accuracy graph")
plt.plot(arr, test_score , label = "train accuracy")
plt.plot(arr, train_score , label = "test accuracy")

plt.legend(loc = "upper left")